In [1]:
import requests
from bs4 import BeautifulSoup 

import pandas as pd

import os
import re
import time
import json

def collect_car_urls(
    brand: str
):
    """
    Collect the all car urls for a specified brand from the auto.ru website. 
    For each brand the number of available pages is calculated and the urls from all these pages are saved 
    into a list.
    """
    main_url = f'https://auto.ru/cars/{brand}/all/'   
    main_response = requests.get(main_url)   
    main_soap = BeautifulSoup(main_response.content.decode('utf-8'), 'html.parser')    
    _ = main_soap.find('span', class_='ButtonWithLoader__content').text.replace(u'\xa0', '')
    urls_total = int(re.findall(r'\d+', _)[0])
    ads_per_page = len(main_soap.find_all('a', class_='Link ListingItemTitle__link'))
    pages_num = urls_total // ads_per_page
#     pages_num = 3 
    
    all_urls = []
    
    for page_num in range(1, pages_num):
        if page_num % 20 == 0:
            print(f"Extracting page {page_num} from {pages_num}...")
        page_url = f'{main_url}?page={page_num}'   
        page_response = requests.get(page_url)
        time.sleep(0.1)
        page_soap = BeautifulSoup(page_response.content.decode('utf-8'), 'html.parser')   
        
        all_urls.extend([a.get('href') for a in page_soap.find_all('a', class_='Link ListingItemTitle__link')])

    return all_urls

In [2]:
def extract_url_data(url: str, extracted_columns: list):
    """
    For the specified url extract information from the auto.ru webpage about all necessary features mentioning
    in the extracted_columns list, such as model_name, mileage, price, etc.
    The function returns the list of values for all specified features in the same order as its fields are 
    presented in the extracted_columns list.
    """
    response = requests.get(url)
    page = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
    #pdb.set_trace()  
    try:
        catalog_url = page.find(
            'a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default').get('href')
        response_catalog = requests.get(catalog_url)
        page_catalog = BeautifulSoup(response_catalog.content.decode('utf-8'), 'html.parser')
    except:
        pass
    try:
        json_data_catalog = json.loads(
            page_catalog.find('script', type="application/json", id='initial-state').string)
    except:
        pass
    try:
        json_data_equip = json.loads(
            page.find('script', type="application/json", id='initial-state').string)
    except:
        pass
    try:
        data = json.loads(
            page.find('script', type="application/ld+json").string)
        data = nd.flatten(data)
    except:
        pass
    
    try:
        data['complectation_dict'] = [
        k for k, v in json_data_catalog['state']['compare']['selected'][0]['options'].items() if v == 1]
    except:
        pass
    try:
        data['equipment_dict'] = json_data_equip['card']['vehicle_info']['equipment']
    except:
        pass
    try:
        data['price'] = data['offers']['price']
    except:
        pass
    try:
        data['priceCurrency'] = data['offers']['priceCurrency']
    except:
        pass
    try:
        mileage = page.find(
            'li', class_='CardInfoRow CardInfoRow_kmAge').find_all('span')[-1].text.replace(u'\xa0', u'')
        data['mileage'] = int(re.findall(r'\d+', mileage)[0])
    except:
        pass
    try:
        data['engineDisplacement'] = data['vehicleEngine']['engineDisplacement']
    except:
        pass
    try:
        data['enginePower'] = data['vehicleEngine']['enginePower']
    except:
        pass
    try:
        data['model_name'] = page.find_all(
        'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
    except:
        pass
    try:
        data['parsing_unixtime'] = int(time.time())
    except:
        pass
    try:
        data['sell_id'] = int(re.findall(
            r'\d+', page.find('div', class_='CardHead__infoItem CardHead__id').text)[0])
    except:
        pass
    try:
        data['super_gen'] = json.loads(
            page.find('div', id="sale-data-attributes").get('data-bem'))
    except:
        pass
    try:
        data['Владельцы'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_ownersCount').find_all('span')[-1].text.replace(u'\xa0', u' ')
    except:
        pass
    try:
        data['Владение'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
    except:
        pass
    try:
        data['ПТС'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text
    except:
        pass
    try:
        data['Привод'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text
    except:
        pass
    try:
        data['Руль'] = page.find('li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text
    except:
        pass
    try:
        data['Состояние'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
    except:
        pass
    try:
        data['Таможня'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
    except:
        pass
    try:
        data['description'] = re.sub('\W+', ' ', data['description'])
    except:
        pass
    
    # additional features
    try:
        data['views'] =  page.find(
            'div', class_='CardHead__infoItem CardHead__views').text.split()[0]
    except:
        pass
    try:
        data['date_added'] = page.find(
            'div', class_='CardHead__infoItem CardHead__creationDate').text 
    except:
        pass
    try:
        data['region'] = page.find(
            'div', class_='CardBreadcrumbs').find_all(
            'div', class_='CardBreadcrumbs__item')[-1].text.replace(u'\xa0', u' ')
    except:
        pass
    try:
        data['car_url'] = url
    except:
        pass
    try:
        data['vendor'] = json_data_equip['card']['vehicle_info']['vendor']
    except:
        pass
    
    
    output = []
    try:
        for col in extracted_columns:
            output.append(data.get(col, None))    
    except:
        pass
    if not output:
        output = [None] * len(extracted_columns)
    return output

In [3]:
# The list of columns we're going to parse
extracted_columns = ['bodyType',
 'brand',
 'car_url',
 'color',
 'complectation_dict',
 'description',
 'engineDisplacement',
 'enginePower',
 'equipment_dict',
 'fuelType',
 'image',
 'mileage',
 'modelDate',
 'model_info',
 'model_name',
 'name',
 'numberOfDoors',
 'parsing_unixtime',
 'priceCurrency',
 'productionDate',
 'sell_id',
 'super_gen',
 'vehicleConfiguration',
 'vehicleTransmission',
 'vendor',
 'Владельцы',
 'Владение',
 'ПТС',
 'Привод',
 'Руль',
 'Состояние',
 'Таможня']

In [4]:
# We will extract data only for brands from the test dataset
brands = [
#  'skoda',
#  'audi',
#  'honda',
#  'volvo',
#  'bmw',
#  'nissan',
#  'infiniti',
#  'mercedes',
#  'toyota',
#  'lexus',
#  'volkswagen',
#  'mitsubishi',
]

# We will extract data for every brand one by one each into separate file
for brand in brands:
    print(f"Extracting data for the brand {brand}:")
    urls = collect_car_urls(brand)
    train_df = pd.DataFrame({'car_url': urls})
    final_list = []
    brand_count = len(train_df.car_url.values)
    for n, url in enumerate(train_df.car_url.values.tolist()[:]):
        print(f"The URL {n + 1} / {brand_count}, url is {url}.")
        try:
            final_list.append(extract_url_data(url, extracted_columns + ['views', 'date_added', 'region', 'price']))
        except Exception as ex:
            print(ex)
            time.sleep(60)
            final_list.append(extract_url_data(url, extracted_columns + ['views', 'date_added', 'region', 'price']))

    df1 = pd.DataFrame(data=final_list, columns=extracted_columns + ['views', 'date_added', 'region', 'price'])

    outdir = './test'
    if not os.path.exists(outdir):
        os.mkdir(outdir)  

    filename = f"df_parsed_{brand}.csv"
    fullname = os.path.join(outdir, filename)  
    df1.to_csv(fullname, index=False)

Extracting data for the brand bmw:
Extracting page 20 from 405...
Extracting page 40 from 405...
Extracting page 60 from 405...
Extracting page 80 from 405...
Extracting page 100 from 405...
Extracting page 120 from 405...
Extracting page 140 from 405...
Extracting page 160 from 405...
Extracting page 180 from 405...
Extracting page 200 from 405...
Extracting page 220 from 405...
Extracting page 240 from 405...
Extracting page 260 from 405...
Extracting page 280 from 405...
Extracting page 300 from 405...
Extracting page 320 from 405...
Extracting page 340 from 405...
Extracting page 360 from 405...
Extracting page 380 from 405...
Extracting page 400 from 405...
The URL 1 / 15203, url is https://auto.ru/cars/used/sale/bmw/x3/1105503111-8480d354/.
The URL 2 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105562690-47d8565a/.
The URL 3 / 15203, url is https://auto.ru/cars/new/group/bmw/x7/21406260/22869441/1104786691-293f3924/.
The URL 4 / 15203, url is https://auto.ru/cars/new/g

The URL 84 / 15203, url is https://auto.ru/cars/used/sale/bmw/7er/1105515560-cf191200/.
The URL 85 / 15203, url is https://auto.ru/cars/used/sale/bmw/x5/1105528680-5923c0e6/.
The URL 86 / 15203, url is https://auto.ru/cars/new/group/bmw/x5/21308274/21519497/1105279928-e04b69cb/.
The URL 87 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1105566806-052a956d/.
The URL 88 / 15203, url is https://auto.ru/cars/used/sale/bmw/x6/1103523767-77929a23/.
The URL 89 / 15203, url is https://auto.ru/cars/used/sale/bmw/x6/1105235602-affec7f2/.
The URL 90 / 15203, url is https://auto.ru/cars/used/sale/bmw/x3/1105178146-0668a9ce/.
The URL 91 / 15203, url is https://auto.ru/cars/used/sale/bmw/x6/1105340798-8b27b0d3/.
The URL 92 / 15203, url is https://auto.ru/cars/used/sale/bmw/x5/1105582792-20c79ba6/.
The URL 93 / 15203, url is https://auto.ru/cars/used/sale/bmw/6er/1105571954-e89d76d9/.
The URL 94 / 15203, url is https://auto.ru/cars/new/group/bmw/x5/21308274/21519497/1105235862-3adc06e9/.
The 

The URL 174 / 15203, url is https://auto.ru/cars/new/group/bmw/x5/21308274/21519497/1105235861-57a244b8/.
The URL 175 / 15203, url is https://auto.ru/cars/used/sale/bmw/x6/1105581438-f9487662/.
The URL 176 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1104998095-bf8c76a1/.
The URL 177 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105303280-0d7c802b/.
The URL 178 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105290682-1199e2db/.
The URL 179 / 15203, url is https://auto.ru/cars/used/sale/bmw/x3/1105515575-a4512e1f/.
The URL 180 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1105102270-43d55814/.
The URL 181 / 15203, url is https://auto.ru/cars/used/sale/bmw/x3/1105551784-e47df9ba/.
The URL 182 / 15203, url is https://auto.ru/cars/used/sale/bmw/x6/1105395141-046c6c0d/.
The URL 183 / 15203, url is https://auto.ru/cars/new/group/bmw/x5/21308274/21519497/1104963022-9048c819/.
The URL 184 / 15203, url is https://auto.ru/cars/new/group/bmw/x5/21308274/22863

The URL 264 / 15203, url is https://auto.ru/cars/used/sale/bmw/x1/1105234011-f443ca32/.
The URL 265 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105447301-61c68fee/.
The URL 266 / 15203, url is https://auto.ru/cars/new/group/bmw/x5/21718431/21935248/1105190903-42ef3b8a/.
The URL 267 / 15203, url is https://auto.ru/cars/used/sale/bmw/x5/1105475524-53ed8df4/.
The URL 268 / 15203, url is https://auto.ru/cars/used/sale/bmw/x3/1105609137-6dac5a08/.
The URL 269 / 15203, url is https://auto.ru/cars/used/sale/bmw/x1/1105470191-661901af/.
The URL 270 / 15203, url is https://auto.ru/cars/used/sale/bmw/7er/1105252362-07ac7397/.
The URL 271 / 15203, url is https://auto.ru/cars/new/group/bmw/x5/21790186/21935216/1105623401-a9ba6c63/.
The URL 272 / 15203, url is https://auto.ru/cars/used/sale/bmw/x4/1104946523-50ca4540/.
The URL 273 / 15203, url is https://auto.ru/cars/used/sale/bmw/x5/1105436322-68129854/.
The URL 274 / 15203, url is https://auto.ru/cars/used/sale/bmw/x6_m/1105497685-256

The URL 354 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1105486876-58dd1ea9/.
The URL 355 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1104979541-205e1b41/.
The URL 356 / 15203, url is https://auto.ru/cars/used/sale/bmw/i3/1105078860-5a9e49f8/.
The URL 357 / 15203, url is https://auto.ru/cars/used/sale/bmw/x6/1105607123-e53b7a5c/.
The URL 358 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1103271676-d9e34759/.
The URL 359 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105566388-4660410e/.
The URL 360 / 15203, url is https://auto.ru/cars/used/sale/bmw/x5/1105259004-b057a655/.
The URL 361 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105066710-ffda7b4a/.
The URL 362 / 15203, url is https://auto.ru/cars/new/group/bmw/x5/21308175/21519385/1105525416-7c549baf/.
The URL 363 / 15203, url is https://auto.ru/cars/used/sale/bmw/x3/1105393120-b13f4713/.
The URL 364 / 15203, url is https://auto.ru/cars/used/sale/bmw/x5/1105250470-70d1eff8/.
The URL 3

...

The URL 2045 / 15203, url is https://auto.ru/cars/used/sale/bmw/x7/1105107294-756820b5/.
The URL 2046 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105629596-8d7d83c3/.
The URL 2047 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1104944075-d703f485/.
The URL 2048 / 15203, url is https://auto.ru/cars/used/sale/bmw/x5_m/1104035080-200e7621/.
The URL 2049 / 15203, url is https://auto.ru/cars/new/group/bmw/x7/22439407/22439743/1105618935-1299d250/.
The URL 2050 / 15203, url is https://auto.ru/cars/new/group/bmw/x7/21731985/21983629/1104963070-0c1e9289/.
The URL 2051 / 15203, url is https://auto.ru/cars/used/sale/bmw/x4/1105572663-28a2c84f/.
The URL 2052 / 15203, url is https://auto.ru/cars/used/sale/bmw/1er/1105094212-6070729a/.
The URL 2053 / 15203, url is https://auto.ru/cars/used/sale/bmw/6er/1104078442-4c8c3bc3/.
The URL 2054 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105594219-4b22c2a6/.
The URL 2055 / 15203, url is https://auto.ru/cars/new/group/bmw/x7/

The URL 2133 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105568620-20668389/.
The URL 2134 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1105629008-453335f9/.
The URL 2135 / 15203, url is https://auto.ru/cars/new/group/bmw/x3/21184620/21593890/1104951471-980645d2/.
The URL 2136 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1104843754-6d26eec7/.
The URL 2137 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105568224-98418f7f/.
The URL 2138 / 15203, url is https://auto.ru/cars/used/sale/bmw/x3/1105591910-ffdcc3d4/.
The URL 2139 / 15203, url is https://auto.ru/cars/used/sale/bmw/x1/1105628878-564795a6/.
The URL 2140 / 15203, url is https://auto.ru/cars/new/group/bmw/x3/21029738/21408289/1104045794-5e9665a4/.
The URL 2141 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1105628808-18605ebf/.
The URL 2142 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105456258-e940cfb8/.
The URL 2143 / 15203, url is https://auto.ru/cars/used/sale/bmw/x4/1

The URL 2222 / 15203, url is https://auto.ru/cars/used/sale/bmw/x3/1105564147-d2a5de39/.
The URL 2223 / 15203, url is https://auto.ru/cars/used/sale/bmw/x3/1105565477-a1934166/.
The URL 2224 / 15203, url is https://auto.ru/cars/used/sale/bmw/7er/1103200159-656d5dfb/.
The URL 2225 / 15203, url is https://auto.ru/cars/new/group/bmw/x3/21029738/21408288/1104150422-239c56e0/.
The URL 2226 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1105564127-bd9dce0e/.
The URL 2227 / 15203, url is https://auto.ru/cars/used/sale/bmw/x1/1105628158-86355458/.
The URL 2228 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105565519-81d79a16/.
The URL 2229 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1105563766-7d15cce9/.
The URL 2230 / 15203, url is https://auto.ru/cars/new/group/bmw/x3/21029738/21408289/1104726137-4dd80c90/.
The URL 2231 / 15203, url is https://auto.ru/cars/used/sale/bmw/6er/1104891484-c375081f/.
The URL 2232 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1

The URL 2311 / 15203, url is https://auto.ru/cars/new/group/bmw/x3/21029738/21408288/1104062977-c6c39599/.
The URL 2312 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1105272756-d370c5ce/.
The URL 2313 / 15203, url is https://auto.ru/cars/used/sale/bmw/x6/1105498537-a814f8ec/.
The URL 2314 / 15203, url is https://auto.ru/cars/used/sale/bmw/x4/1105406860-591eb15e/.
The URL 2315 / 15203, url is https://auto.ru/cars/used/sale/bmw/3er/1105362505-6b1df826/.
The URL 2316 / 15203, url is https://auto.ru/cars/used/sale/bmw/x3/1105444469-d30d3741/.
The URL 2317 / 15203, url is https://auto.ru/cars/new/group/bmw/x3/21029738/21408288/1104635766-4df6e456/.
The URL 2318 / 15203, url is https://auto.ru/cars/used/sale/bmw/5er/1105399968-ecfbfb44/.
The URL 2319 / 15203, url is https://auto.ru/cars/used/sale/bmw/4/1105518962-cdf47aeb/.
The URL 2320 / 15203, url is https://auto.ru/cars/used/sale/bmw/x5/1105265483-edb9fb07/.
The URL 2321 / 15203, url is https://auto.ru/cars/new/group/bmw/x3/21029

KeyboardInterrupt: 